### Load tensorflow

In [0]:
# Load all necessary Libraries
import tensorflow as tf
import pandas as pd

In [0]:
#Reset Default graph - Needed only for Jupyter notebook
tf.reset_default_graph()

### Collect Data

In [0]:
# Loading the dataset
price = pd.read_csv('./prices.csv')

### Check all columns in the dataset

In [4]:
#Checking all the columns
price.columns

Index([u'date', u'symbol', u'open', u'close', u'low', u'high', u'volume'], dtype='object')

In [5]:
# Checking the columns of the dataset
price.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [6]:
# info about the dataset
price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88890 entries, 0 to 88889
Data columns (total 7 columns):
date      88890 non-null object
symbol    88889 non-null object
open      88889 non-null float64
close     88889 non-null float64
low       88889 non-null float64
high      88889 non-null float64
volume    88889 non-null float64
dtypes: float64(5), object(2)
memory usage: 4.7+ MB


### Drop columns `date` and  `symbol`

In [0]:
# Dropping date & symbol column from the dataset
price.drop(['date','symbol'], axis=1, inplace=True)


In [8]:
# After dropping the columns 
price.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [9]:
# first 1000 rows for building the feature and target set
pricedf= price.head(1000)
pricedf.shape

(1000, 5)

### Divide the data into train and test sets

In [10]:

# Dependent Variable
Y = pricedf['close']

# Independent Variable
X = pricedf[['open','high','low','volume']]

print("Shape of Dependent Variable:\n", Y.shape,"\n")
print("Shape of Independent Variable:\n", X.shape)

('Shape of Dependent Variable:\n', (1000,), '\n')
('Shape of Independent Variable:\n', (1000, 4))


In [11]:
from sklearn.model_selection import train_test_split
# Dividing the dataset into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)
# random state is setting up a seed value

# Shape of training and tst feature set
print("Shape of X Train:\t",X_train.shape)
print("Shape of X Test:\t",X_test.shape)
print("Shape of Y Train:\t",Y_train.shape)
print("Shape of Y Test:\t",Y_test.shape)

('Shape of X Train:\t', (700, 4))
('Shape of X Test:\t', (300, 4))
('Shape of Y Train:\t', (700,))
('Shape of Y Test:\t', (300,))


## Building the graph in tensorflow

1.Define input data placeholders

In [0]:
#Input features
x = tf.placeholder(shape=[None,4],dtype=tf.float32, name='x-input')

#Normalize the data
x_n = tf.layers.batch_normalization(x,training=True)

#Actual Volume
y_ = tf.placeholder(shape=[None],dtype=tf.float32, name='y-input')

2.Define Weights and Bias

In [0]:
W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]),name="Bias")

3.Prediction

In [0]:
#We will use normalized data

y = tf.add(tf.matmul(x_n,W),b,name='output')

4.Loss (Cost) Function [Mean square error]

In [0]:
loss = tf.reduce_mean(tf.square(y-y_),name='Loss')

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [0]:
train_op = tf.train.GradientDescentOptimizer(0.003).minimize(loss)

## Execute the Graph for 100 epochs and observe the loss

In [0]:
#Lets start graph Execution
sess = tf.Session()

# variables need to be initialized before we can use them
sess.run(tf.global_variables_initializer())

#how many times data need to be shown to model
training_epochs = 100

In [18]:
for epoch in range(100):
            
    #Calculate train_op and loss
    _, train_loss = sess.run([train_op,loss],feed_dict={x:X_train, 
                                                        y_:Y_train})
    
    if epoch % 10 == 0:
        print ('Training loss at step: ', epoch, ' is ', train_loss)

('Training loss at step: ', 0, ' is ', 7617.3755)
('Training loss at step: ', 10, ' is ', 7101.4033)
('Training loss at step: ', 20, ' is ', 6643.554)
('Training loss at step: ', 30, ' is ', 6238.0874)
('Training loss at step: ', 40, ' is ', 5876.771)
('Training loss at step: ', 50, ' is ', 5298.895)
('Training loss at step: ', 60, ' is ', 3067.3862)
('Training loss at step: ', 70, ' is ', 3065.9136)
('Training loss at step: ', 80, ' is ', 3065.8994)
('Training loss at step: ', 90, ' is ', 3065.8887)


In [19]:
sess.run(y, feed_dict={x:X_train[0:1]})

array([[67.383934]], dtype=float32)

In [20]:
#test
for epoch in range(100):
            
    #Calculate train_op and loss
    _, test_loss = sess.run([train_op,loss],feed_dict={x:X_test, 
                                                        y_:Y_test})
    
    if epoch % 10 == 0:
        print ('Training loss at step: ', epoch, ' is ', test_loss)

('Training loss at step: ', 0, ' is ', 4189.73)
('Training loss at step: ', 10, ' is ', 4184.4434)
('Training loss at step: ', 20, ' is ', 4184.4365)
('Training loss at step: ', 30, ' is ', 4184.433)
('Training loss at step: ', 40, ' is ', 4184.4297)
('Training loss at step: ', 50, ' is ', 4184.424)
('Training loss at step: ', 60, ' is ', 4184.4194)
('Training loss at step: ', 70, ' is ', 4184.416)
('Training loss at step: ', 80, ' is ', 4184.416)
('Training loss at step: ', 90, ' is ', 4184.413)


In [21]:
sess.run(y, feed_dict={x:X_test[0:1]})

array([[65.12346]], dtype=float32)

### Get the shapes and values of W and b

Hint: Use sess.run(W) to get W.

In [22]:
sess.run(W)

array([[-1.5369328 ],
       [ 6.259404  ],
       [-2.1015074 ],
       [ 0.14077675]], dtype=float32)

In [23]:
sess.run(b)

array([19.076443], dtype=float32)

### Find the Absolute mean square loss difference between training and testing loss.

In [0]:
error= test_loss-train_loss

In [25]:
error

1118.5264

### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [0]:
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.003)

In [0]:
#Initialize Sequential Graph (model)
model = tf.keras.models.Sequential()

#Normalize input data
model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))

#Add Dense layer for prediction - Keras declares weights and bias automatically
model.add(tf.keras.layers.Dense(1))

#Compile the model - add Loss and Gradient Descent optimizer
model.compile(optimizer=sgd_optimizer, loss='mse')

### Execute the model

In [28]:
model.fit(X_train, Y_train, epochs=100)

Epoch 1/100
700/700 [==============================] - 0s 219us/step - loss: 2191.7491
Epoch 2/100
700/700 [==============================] - 0s 36us/step - loss: 968.8051
Epoch 3/100
700/700 [==============================] - 0s 33us/step - loss: 924.1359
Epoch 4/100
700/700 [==============================] - 0s 32us/step - loss: 1160.1395
Epoch 5/100
700/700 [==============================] - 0s 32us/step - loss: 704.9167
Epoch 6/100
700/700 [==============================] - 0s 30us/step - loss: 996.6610
Epoch 7/100
700/700 [==============================] - 0s 36us/step - loss: 802.7510
Epoch 8/100
700/700 [==============================] - 0s 35us/step - loss: 1491.9614
Epoch 9/100
700/700 [==============================] - 0s 34us/step - loss: 977.1567
Epoch 10/100
700/700 [==============================] - 0s 39us/step - loss: 1019.9182
Epoch 11/100
700/700 [==============================] - 0s 31us/step - loss: 642.1945
Epoch 12/100
700/700 [==============================] - 0s

In [0]:
sess.close()

### Classification using Keras 

### Load the given Iris data using pandas (Iris.csv)

In [0]:
# Loading the dataset
iris = pd.read_csv('/content/Iris1.csv')

In [31]:
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Splitting the data into feature set and target set

In [0]:
iris.drop(['Id'], axis=1, inplace=True)

In [33]:

# Dependent Variable
Y = iris['Species']

# Independent Variable
X = iris[['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']]

print("Shape of Dependent Variable:\n", Y.shape,"\n")
print("Shape of Independent Variable:\n", X.shape)

('Shape of Dependent Variable:\n', (150,), '\n')
('Shape of Independent Variable:\n', (150, 4))


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
from sklearn.preprocessing import LabelEncoder

In [35]:
Split_Y = pd.DataFrame(Y).apply(LabelEncoder().fit_transform)
Split_Y.head()

,Species
0,0
1,0
2,0
3,0
4,0


In [36]:
trans_Y= tf.keras.utils.to_categorical(Split_Y, num_classes=3)
trans_Y.shape

(150, 3)

### Divide the dataset into Training and test (70:30)

In [37]:
from sklearn.model_selection import train_test_split
# Dividing the dataset into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, trans_Y, test_size=0.3)
# random state is setting up a seed value

# Shape of training and tst feature set
print("Shape of X Train:\t",X_train.shape)
print("Shape of X Test:\t",X_test.shape)
print("Shape of Y Train:\t",Y_train.shape)
print("Shape of Y Test:\t",Y_test.shape)

('Shape of X Train:\t', (105, 4))
('Shape of X Test:\t', (45, 4))
('Shape of Y Train:\t', (105, 3))
('Shape of Y Test:\t', (45, 3))


### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [0]:
#Initialize Sequential Graph (model)
model = tf.keras.models.Sequential()

#Normalize input data
model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))

#Add 1st hidden layer
model.add(tf.keras.layers.Dense(10, activation='sigmoid'))

#Add 2nd hidden layer
model.add(tf.keras.layers.Dense(8, activation='sigmoid'))

#Add OUTPUT layer
model.add(tf.keras.layers.Dense(3, activation='softmax'))

In [0]:
#Create optimizer with non-default learning rate
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)

#Compile the model
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

### Fitting the model and predicting 

In [40]:
model.fit(X_train,Y_train,          
          validation_data=(X_test,Y_test),
          epochs=100,
          batch_size=50)

Train on 105 samples, validate on 45 samples
Epoch 1/100
105/105 [==============================] - 0s 3ms/step - loss: 1.2041 - acc: 0.0857 - val_loss: 1.2132 - val_acc: 0.3111
Epoch 2/100
105/105 [==============================] - 0s 110us/step - loss: 1.1980 - acc: 0.0952 - val_loss: 1.2114 - val_acc: 0.3111
Epoch 3/100
105/105 [==============================] - 0s 99us/step - loss: 1.1966 - acc: 0.3333 - val_loss: 1.2109 - val_acc: 0.3111
Epoch 4/100
105/105 [==============================] - 0s 106us/step - loss: 1.1940 - acc: 0.3429 - val_loss: 1.2031 - val_acc: 0.3111
Epoch 5/100
105/105 [==============================] - 0s 133us/step - loss: 1.1891 - acc: 0.3429 - val_loss: 1.1862 - val_acc: 0.3111
Epoch 6/100
105/105 [==============================] - 0s 128us/step - loss: 1.1801 - acc: 0.2667 - val_loss: 1.1866 - val_acc: 0.3111
Epoch 7/100
105/105 [==============================] - 0s 109us/step - loss: 1.1805 - acc: 0.3429 - val_loss: 1.1772 - val_acc: 0.3111
Epoch 8/100
1

### Report Accuracy of the predicted values

In [0]:
model.save('iris_dnn_v1.h5')

In [42]:
!ls

Iris1.csv  iris_dnn_v1.h5  prices.csv  sample_data


In [0]:
pred_model = tf.keras.models.load_model('iris_dnn_v1.h5')

In [44]:
model.evaluate(X_test,Y_test)

45/45 [==============================] - 0s 170us/step


[1.0689403110080296, 0.5111111190583971]

In [45]:
X_test[0:1].shape

(1, 4)

In [46]:
#Actual Label for the first example in test dataset
print(Y_test[0])

#Model prediction
prediction = pred_model.predict(X_test[0:1])

#Print prediction
print(prediction[0])

[0. 0. 1.]
[0.32082626 0.36280182 0.31637195]
